# Fig 6: Onset-Offset Comparisons
### Comparing Within-Detector Sensitivity to Onset vs. Offset (Fixations & Saccades Separately)

In [20]:
import copy
import warnings
from typing import Optional

import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import scikit_posthocs as sp
import scipy.stats as st
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
from statsmodels import formula

import peyes

from analysis._article_results.lund2013._helpers import *
import analysis.statistics.channel_sdt as ch_sdt

# pio.renderers.default = "browser"

### Set Constants

In [21]:
THRESHOLD = 5       # temporal threshold for analyzing d'
METRIC = peyes.constants.D_PRIME_STR
ANNOTATOR = 'RA'

GRID_LINE_COLOR, GRID_LINE_WIDTH = "lightgray", 1
ZERO_LINE_WIDTH = 2 * GRID_LINE_WIDTH

FONT_FAMILY, FONT_COLOR = "Calibri", "black"
TITLE_FONT = dict(family=FONT_FAMILY, size=25, color=FONT_COLOR)
SUBTITLE_FONT = dict(family=FONT_FAMILY, size=20, color=FONT_COLOR)
AXIS_LABEL_FONT = dict(family=FONT_FAMILY, size=20, color=FONT_COLOR)
AXIS_TICK_FONT = dict(family=FONT_FAMILY, size=18, color=FONT_COLOR)
AXIS_LABEL_STANDOFF = 2

## Load Data

In [22]:
fix_metrics = ch_sdt.load(
    dataset_name=DATASET_NAME,
    output_dir=PROCESSED_DATA_DIR,
    label=1,    # EventLabelEnum.FIXATION.value
    stimulus_type=STIMULUS_TYPE,
    threshold=THRESHOLD,
    channel_type=None,
)

sac_metrics = ch_sdt.load(
    dataset_name=DATASET_NAME,
    output_dir=PROCESSED_DATA_DIR,
    label=2,    # EventLabelEnum.SACCADE.value
    stimulus_type=STIMULUS_TYPE,
    threshold=THRESHOLD,
    channel_type=None,
)

# Remove unused metrics
fix_metrics.drop(index=['P', 'PP', 'N', 'TP'], level=peyes.constants.METRIC_STR, inplace=True)
sac_metrics.drop(index=['P', 'PP', 'N', 'TP'], level=peyes.constants.METRIC_STR, inplace=True)

# concatenate
metrics = pd.concat([fix_metrics, sac_metrics], keys=['fixation', 'saccade'])
metrics.index.names = [peyes.constants.EVENT_STR] + metrics.index.names[1:]
metrics = metrics.droplevel('threshold')

metrics

trial_id                                      25                      \
gt                                            RA                       
pred                                          MN   engbert  remodnav   
event    channel_type metric                                           
fixation onset        recall            0.785714  0.964286  0.285714   
                      precision         0.814815  0.818182  0.666667   
                      f1                0.800000  0.885246  0.400000   
                      false_alarm_rate  0.011752  0.014103  0.009402   
                      d_prime           3.056776  3.997164  1.783457   
                      criterion         0.736749  0.195839  1.457677   
         offset       recall            0.928571  1.000000  0.142857   
                      precision         0.962963  0.848485  0.333333   
                      f1                0.945455  0.918033  0.200000   
                      false_alarm_rate  0.002350  0.011752  0.018803   
                      d_prime           4.292082  5.048805  1.011544   
                      criterion         0.680807 -0.324197  1.573342   
saccade  onset        recall            1.000000  1.000000  0.520000   
                      precision         0.961538  0.806452  1.000000   
                      f1                0.980392  0.892857  0.684211   
                      false_alarm_rate  0.002334  0.014004  0.000000   
                      d_prime           5.459188  4.989576  2.898532   
                      criterion        -0.119005 -0.353811  1.399333   
         offset       recall            0.920000  0.520000  0.120000   
                      precision         0.884615  0.419355  0.230769   
                      f1                0.901961  0.464286  0.157895   
                      false_alarm_rate  0.007002  0.042011  0.023340   
                      d_prime           3.862237  1.777960  0.814211   
                      criterion         0.526047  0.838826  1.582092   

trial_id                                                              \
gt                                                                     
pred                                        idvt        nh       idt   
event    channel_type metric                                           
fixation onset        recall            0.035714  0.678571  0.035714   
                      precision         0.055556  0.703704  0.055556   
                      f1                0.043478  0.690909  0.043478   
                      false_alarm_rate  0.039957  0.018803  0.039957   
                      d_prime          -0.051561  2.542822 -0.051561   
                      criterion         1.776963  0.807703  1.776963   
         offset       recall            0.571429  0.500000  0.571429   
                      precision         0.888889  0.518519  0.888889   
                      f1                0.695652  0.509091  0.695652   
                      false_alarm_rate  0.004701  0.030556  0.004701   
                      d_prime           2.777103  1.872691  2.777103   
                      criterion         1.208539  0.936345  1.208539   
saccade  onset        recall            0.640000  0.480000  0.640000   
                      precision         0.941176  0.461538  0.941176   
                      f1                0.761905  0.470588  0.761905   
                      false_alarm_rate  0.002334  0.032676  0.002334   
                      d_prime           3.187557  1.792695  3.187557   
                      criterion         1.235320  0.946501  1.235320   
         offset       recall            0.160000  0.600000  0.160000   
                      precision         0.235294  0.576923  0.235294   
                      f1                0.190476  0.588235  0.190476   
                      false_alarm_rate  0.030342  0.025674  0.030342   
                      d_prime           0.881339  2.201913  0.881339   
                      criterion        

## Functions
### Linear Mixed Effects Model
We have a measurement (e.g., $d'$) for a hierarchy of conditions: Dataset (entire population) $\rightarrow$ GT Annotator $\rightarrow$ PRED Detector $\rightarrow$ Event (fixation/saccade) $\rightarrow$ Channel (onset/offset) $\rightarrow$ single (trial) measurement.  
  
**(1) Between-Detector Comparison:**  
For each **annotator** _(RA, MN)_ separately, we want to test the effect of **channel** _(onset/offset)_ and **event** _(fixation/saccade)_ and their interaction, across all **detectors**, on the measurement. We can do this using a nested linear mixed effects model (LME), using the `statsmodels` package. To put this in formula form:  
$$d' \sim 1 + \text{event} + \text{channel} + \text{event} \times \text{channel} + (1|\text{detector})$$
where the `(1|detector)` term specifies that the detector is a random effect.  
  
  
**(2) Within-Detector Comparison:**
For each **annotator-detector** pair, we want to test the effect of **event** _(fixation/saccade)_ on the measurement. To put this in formula form:
$$d' \sim 1 + \text{event} + (1|\text{channel})$$
where the `(1|channel)` term specifies that the channel (onset/offset) is a random effect.

In [23]:
def linear_mixed_effect(
        dataset: pd.DataFrame, metric: str, gt_annotator: str, pred_detector: Optional[str] = None, include_annotators: bool = True
):
    # extract the subset of data and reshape it from "wide" to "long" format
    subset = _extract_data(dataset, metric, gt_annotator, pred_detector, include_annotators)
    long_subset = _reshape_data(subset, metric)
    
    # create the LME model and fit to the long-format data
    if pred_detector:
        # within-detector comparison, use fixation/saccade as the fixed effect and onset/offset as the grouping variable
        formula = f"{metric} ~ {peyes.constants.EVENT_STR}"     # fixed effect: fixation/saccade
        groups = peyes.constants.CHANNEL_TYPE_STR               # grouping variable: onset/offset
    else:
        # between-detector comparison, use fixation/saccade and onset/offset and their interaction as the fixed effect and the detector as grouping variable
        formula = f"{metric} ~ {peyes.constants.EVENT_STR} + {peyes.constants.CHANNEL_TYPE_STR} + {peyes.constants.EVENT_STR} * {peyes.constants.CHANNEL_TYPE_STR}"     # fixed effect: fixation/saccade, onset/offset, and their interaction
        groups = u.PRED_STR                # grouping variable: detector
    
    # add random intercept per group (detectors/events) and random slopes w.r.t. each fixed effect (events, channels)
    re_formula = "1 + " + formula.split(" ~ ")[1]
    
    # create and fit the model
    model = smf.mixedlm(formula, long_subset, groups=groups, re_formula=re_formula)
    result = model.fit()
    return result

def _extract_data(
        dataset: pd.DataFrame, metric: str, gt_annotator: str, pred_detector: Optional[str] = None, include_annotators: bool = True
) -> pd.DataFrame:
    """
    Extracts data for the given metric, GT annotator, and PRED detector, if provided.
    If pred_detector is None, the function returns the data for all detectors, including the GT annotators, unless `include_annotators` is set to False.
    
    dataset structure:
    - index contains the following levels: "event" (fixation/saccade), "channel_type" (onset/offset), "metric" (e.g. d_prime, f1, etc.)
    - columns contain the following levels: "trial_id" (numeric: 25, 26...), "gt" (RA/MN), "pred" (algorithmic detector)
    
    Returns a DataFrame with the following structure:  
    - Rows: MultiIndex with levels [pred, event, channel_type] if pred_detector is None, otherwise [event, channel_type]
    - Columns: Single-level index with trial_id
    - Values are the metric values for each trial for the (pred, event, channel_type) combination.
    
    Raise an AssertionError if the metric/annotator/detector is not found in the dataset.
    """
    all_metrics = dataset.index.get_level_values(peyes.constants.METRIC_STR).unique()
    assert metric in all_metrics, f"Metric '{metric}' not in {all_metrics}"
    all_annotators = dataset.columns.get_level_values(u.GT_STR).unique()
    assert gt_annotator in all_annotators, f"Annotator '{gt_annotator}' not in {all_annotators}"
    result = dataset.xs(metric, level=peyes.constants.METRIC_STR, axis=0, drop_level=True).xs(gt_annotator, level=u.GT_STR, axis=1, drop_level=True)
    result = result.stack(level=u.PRED_STR, future_stack=True).reorder_levels([u.PRED_STR, peyes.constants.EVENT_STR, peyes.constants.CHANNEL_TYPE_STR])
    result.sort_index(inplace=True)
    if not include_annotators:
        is_annotator = result.index.get_level_values(u.PRED_STR).isin(all_annotators)
        result = result.loc[~is_annotator]
    
    # return the full subset if pred_detector is None
    if not pred_detector:
        return result
    # return the subset for the given pred_detector
    all_detectors = result.index.get_level_values(u.PRED_STR).unique()
    if pred_detector in all_detectors:
        return result.xs(pred_detector, level=u.PRED_STR, axis=0, drop_level=True)
    if pred_detector == "mean":
        return result.groupby(level=[peyes.constants.EVENT_STR, peyes.constants.CHANNEL_TYPE_STR]).mean()
    raise AssertionError(f"Unidentified Detector '{pred_detector}'")


def _reshape_data(data: pd.DataFrame, metric: str) -> pd.DataFrame:
    """ Reshape a DataFrame from "wide" to "long" format, using the metric as the value variable. """
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=FutureWarning)
        index_names, column_name = data.index.names, data.columns.name
        long_format_data = data.reset_index().melt(
            id_vars=index_names,
            var_name=column_name,
            value_name=metric
        )
    long_format_data.dropna(axis=0, inplace=True)   # remove rows with missing values
    return long_format_data

### Post-Hoc Dunn's Test
#### Examining detection differences **between** Detectors or **within** a Detector 
We compare detection performance between different event types (fixations/saccades) and channel types (onset/offset), across all detectors or for a specific detector (based on whether the `pred_detector` argument is provided or not). We use Dunn's post-hoc test to calculate pairwise differences, with the Bonferroni correction for multiple comparisons. The reason we use Dunn's test is that it is a non-parametric test that does not assume normality, and it is suitable for comparing multiple groups.

In [24]:
def posthoc_dunn(
        dataset, metric: str, gt_annotator: str, pred_detector: Optional[str], include_annotators: bool = True
) -> pd.DataFrame:
    pvalues = _calculate_pvalues(dataset, metric, gt_annotator, pred_detector, include_annotators)
    table = _fill_post_hoc_table(pvalues)
    if pred_detector:
        index_names = [peyes.constants.EVENT_STR, peyes.constants.CHANNEL_TYPE_STR]
    else:
        index_names = [u.PRED_STR, peyes.constants.EVENT_STR, peyes.constants.CHANNEL_TYPE_STR]
    new_index = pd.MultiIndex.from_tuples(table.index.map(lambda idx: tuple(idx.split('_'))), names=index_names)
    table.index = table.columns = new_index
    return table


def _calculate_pvalues(
        data: pd.DataFrame, metric: str, gt_annotator: str, pred_detector: Optional[str], include_annotators: bool = True
) -> pd.DataFrame:
    subset = _extract_data(data, metric, gt_annotator, pred_detector, include_annotators)
    long_subset = _reshape_data(subset, metric)
    if pred_detector:
        # within-detector comparison
        new_colname = f"{peyes.constants.EVENT_STR}_{peyes.constants.CHANNEL_TYPE_STR}"
        long_subset[new_colname] = long_subset[peyes.constants.EVENT_STR] + '_' + long_subset[peyes.constants.CHANNEL_TYPE_STR]
    else:
        # between-detector comparison
        new_colname = f"{u.PRED_STR}_{peyes.constants.EVENT_STR}_{peyes.constants.CHANNEL_TYPE_STR}"
        long_subset[new_colname] = long_subset[u.PRED_STR] + '_' + long_subset[peyes.constants.EVENT_STR] + '_' + long_subset[peyes.constants.CHANNEL_TYPE_STR]
    pvalues = sp.posthoc_dunn(long_subset, val_col=metric, group_col=new_colname, p_adjust='bonferroni')
    return pvalues


def _fill_post_hoc_table(
        posthoc_pvals: pd.DataFrame, alpha: float = 0.05, marginal_alpha: Optional[float] = 0.075,
) -> pd.DataFrame:
    assert 0 < alpha < 1, f"parameter `alpha` must be in range (0, 1), {alpha: .3f} given."
    if marginal_alpha is not None:
        assert alpha < marginal_alpha < 1, f"parameter `marginal_alpha` must be in range ({alpha: .3f}, 1), {marginal_alpha: .3f} given."
    table = np.full_like(posthoc_pvals, "n.s.", dtype=np.dtypes.StringDType())
    if marginal_alpha is not None:
        table[posthoc_pvals <= marginal_alpha] = '†'
    table[posthoc_pvals <= alpha] = '*'
    table[posthoc_pvals <= alpha / 5] = '**'
    table[posthoc_pvals <= alpha / 50] = '***'
    table = pd.DataFrame(table, index=posthoc_pvals.index, columns=posthoc_pvals.columns)
    
    for i, idx in enumerate(table.index):
        for j, col in enumerate(table.columns):
            if i == j:
                table.loc[idx, col] = '--'
            if i > j:
                table.iloc[i, j] = posthoc_pvals.iloc[j, i]
    return table

### Plotting

In [25]:
def single_bar_plot(
        data: pd.DataFrame, metric: str, gt_annotator: str, pred_detector: Optional[str] = None, include_annotators: bool = True
) -> go.Figure:
    subset = _extract_data(data, metric, gt_annotator, pred_detector, include_annotators)
    if not pred_detector:
        subset = subset.groupby(level=[peyes.constants.EVENT_STR, peyes.constants.CHANNEL_TYPE_STR]).mean()     # average across detectors
    summary = pd.concat([subset.mean(axis=1).rename("mean"), subset.std(axis=1).rename("std")], axis=1).reset_index()
    category_orders = {peyes.constants.CHANNEL_TYPE_STR: ["onset", "offset"]}   # order of the bars
    fig = px.bar(
        summary,
        x=peyes.constants.EVENT_STR, y="mean", error_y="std",
        color=peyes.constants.CHANNEL_TYPE_STR, category_orders=category_orders,
        labels={peyes.constants.EVENT_STR: "Event Type", "mean": "Mean", "std": "Standard Deviation"},
        barmode='group',
    )

    # update layout
    fig.for_each_annotation(lambda ann: ann.update(font=SUBTITLE_FONT, textangle=0,))
    fig.update_xaxes(
        title=dict(font=AXIS_LABEL_FONT, standoff=AXIS_LABEL_STANDOFF),
        showline=False,
        showgrid=False, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
        zeroline=False, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
        tickfont=AXIS_TICK_FONT,
    )
    fig.update_yaxes(
        title=dict(text=metric.replace('_', ' ').title(), font=AXIS_LABEL_FONT, standoff=AXIS_LABEL_STANDOFF),
        showline=False,
        showgrid=True, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
        zeroline=True, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
        tickfont=AXIS_TICK_FONT,
    )
    fig.update_layout(
        font_family=FONT_FAMILY,
        width=800, height=450,
        paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',
        margin=dict(l=0, r=0, t=40, b=0, pad=0),
        title=dict(text=f"Fixation vs. Saccade (GT: {gt_annotator})", font=TITLE_FONT),
    )
    return fig


def multiple_bar_plots(data: pd.DataFrame, metric: str, gt_annotator: str, show_other_annotator: bool = True) -> go.Figure:
    subset = _extract_data(data, metric, gt_annotator, include_annotators=show_other_annotator)
    detectors = sorted(
        subset.index.get_level_values(u.PRED_STR).unique(),
        key=lambda dett: LABELER_PLOTTING_CONFIG[dett][0]
    )
    overall_rows = 2
    detector_rows = 2 if len(detectors) > 4 else 1
    detector_cols = int(np.ceil(len(detectors) / detector_rows))
    subtitle = ["Overall"] + list(map(_rename_detector, detectors))
    final_fig = make_subplots(
        rows=2 + detector_rows, cols=detector_cols, shared_xaxes=True, shared_yaxes=True,
        subplot_titles=subtitle,
        vertical_spacing=0.075, horizontal_spacing=0.025,
        specs=[
            [{"type": "bar", "rowspan": overall_rows, "colspan": detector_cols}, *[None for _c in range(detector_cols - 1)]],
            [None for _c in range(detector_cols)],
            *[[{"type": "bar"}for _c in range(detector_cols)] for _r in range(detector_rows)],
        ],
        print_grid=False,
    )
    
    # per-detector plots
    for i, det in enumerate(detectors):
        row, col = 1 + overall_rows + i // detector_cols, 1 + i % detector_cols
        bar_fig = single_bar_plot(data, metric, gt_annotator, det, include_annotators=True)
        bar_fig.for_each_trace(lambda trace: final_fig.add_trace(trace, row=row, col=col))
    final_fig.for_each_trace(lambda trace: trace.update(showlegend=False))

    # overall plot
    global_bar_fig = single_bar_plot(data, metric, gt_annotator, include_annotators=False)
    global_bar_fig.for_each_trace(lambda trace: final_fig.add_trace(trace, row=1, col=1))
    
    # update layout
    final_fig.for_each_annotation(lambda ann: ann.update(font=SUBTITLE_FONT, textangle=0,))
    final_fig.update_xaxes(
        title=dict(font=AXIS_LABEL_FONT, standoff=AXIS_LABEL_STANDOFF),
        showline=False,
        showgrid=False, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
        zeroline=False, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
        tickfont=AXIS_TICK_FONT,
    )
    final_fig.update_yaxes(
        title=dict(font=AXIS_LABEL_FONT, standoff=AXIS_LABEL_STANDOFF),
        showline=False,
        showgrid=True, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
        zeroline=True, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
        tickfont=AXIS_TICK_FONT,
    )
    for r in range(detector_rows):
        final_fig.update_yaxes(row=1 + overall_rows + r, col=1, title_text=metric.replace('_', ' ').title(),)
    final_fig.update_layout(
        font_family=FONT_FAMILY,
        width=900, height=750,
        paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',
        margin=dict(l=0, r=0, t=35, b=0, pad=0),
        legend=dict(
            orientation="h", yanchor="top", xanchor="center", xref='paper', yref='paper', x=0.12, y=0.98,
            font=AXIS_LABEL_FONT, itemwidth=30,
        ),
        title=dict(text=f"Fixation vs. Saccade (GT: <i>{gt_annotator}</i>)", font=TITLE_FONT),
    )
    return final_fig

def _rename_detector(det: str) -> str:
    if det in [GT1, GT2]:
        return f"<i>Ann. {det.upper()}</i>"
    if det.lower() == "remodnav":
        return "REMoDNaV"
    if det.lower().startswith("i"):
        return det.replace("i", "i-").upper()
    return det.upper()

## Results

### (0) Difference in Means
Check if there is a difference in mean performance across all detectors and trials.

In [26]:
_extract_data(metrics, METRIC, ANNOTATOR, include_annotators=False).groupby(
    level=[peyes.constants.EVENT_STR, peyes.constants.CHANNEL_TYPE_STR]).mean().mean(axis=1).unstack(1)

channel_type,offset,onset
event,,
fixation,2.785907,1.597712
saccade,1.600489,3.034778


### (1) Between-Detector Comparison
Exclude the 2nd annotator (_RA_ or _MN_) to make sure we check for differences among algorithms, excluding humans

#### (1A) Linear Mixed Effects Model
We use the full model, controlling for variability between detectors:
$$d' \sim 1 + \text{event} + \text{channel} + \text{event} \times \text{channel} + (1|\text{detector})$$

In [27]:
global_results = linear_mixed_effect(metrics, METRIC, ANNOTATOR, include_annotators=False)
global_results.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                                    Mixed Linear Model Regression Results
=============================================================================================================
Model:                              MixedLM                   Dependent Variable:                   d_prime  
No. Observations:                   560                       Method:                               REML     
No. Groups:                         7                         Scale:                                0.5993   
Min. group size:                    80                        Log-Likelihood:                       -688.2611
Max. group size:                    80                        Converged:                            Yes      
Mean group size:                    80.0                                                                     
-------------------------------------------------------------------------------------------------------------
                                                                   Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------------------------------------
Intercept                                                           2.786    0.462  6.031 0.000  1.881  3.691
event[T.saccade]                                                   -1.185    0.377 -3.148 0.002 -1.924 -0.447
channel_type[T.onset]                                              -1.188    0.431 -2.755 0.006 -2.034 -0.343
event[T.saccade]:channel_type[T.onset]                              2.622    0.650  4.036 0.000  1.349  3.896
pred Var                                                            1.463    1.121                           
pred x event[T.saccade] Cov                                        -0.899    0.812                           
event[T.saccade] Var                                                0.933    0.745                           
pred x channel_type[T.onset] Cov                                   -0.660    0.823                           
event[T.saccade] x channel_type[T.onset] Cov                        0.884    0.773                           
channel_type[T.onset] Var                                           1.243    0.977                           
pred x event[T.saccade]:channel_type[T.onset] Cov                   1.212    1.292                           
event[T.saccade] x event[T.saccade]:channel_type[T.onset] Cov      -1.539    1.243                           
channel_type[T.onset] x event[T.saccade]:channel_type[T.onset] Cov -1.732    1.409                           
event[T.saccade]:channel_type[T.onset] Var                          2.835    2.217                           
=============================================================================================================

"""

In [28]:
results_table = pd.concat(
    [
        global_results.params.map(lambda b: f"{b :.3f}"),
        global_results.tvalues.map(lambda t: f"{t :.3f}"),
        global_results.pvalues.map(lambda p: f"{p :.3f}" if p>=ALPHA / 50 else f"<{ALPHA / 50 :.3f}")],
    keys=['β', 't', 'p'], axis=1
)
is_sig = global_results.pvalues.map(
    lambda p: "***" if p <= ALPHA / 50 else "**" if p <= ALPHA / 5 else "*" if p <= ALPHA else "†" if p <= MARGINAL_ALPHA else "n.s."
).rename("is sig")
conf_invl = global_results.conf_int(ALPHA).rename(columns={0: f"[{ALPHA/2 :.3f}", 1: f"{1 - ALPHA/2 :.3f}]"}).map(lambda val: f"{val: .3f}")

results_table = pd.concat([results_table, is_sig, conf_invl], axis=1)
results_table

,β,t,p,is sig,[0.025,0.975]
Intercept,2.786,6.031,<0.001,***,1.881,3.691
event[T.saccade],-1.185,-3.148,0.002,**,-1.924,-0.447
channel_type[T.onset],-1.188,-2.755,0.006,**,-2.034,-0.343
event[T.saccade]:channel_type[T.onset],2.622,4.036,<0.001,***,1.349,3.896
pred Var,2.442,1.687,0.092,n.s.,-0.396,5.280
pred x event[T.saccade] Cov,-1.500,-1.431,0.153,n.s.,-3.555,0.555
event[T.saccade] Var,1.557,1.617,0.106,n.s.,-0.330,3.443
pred x channel_type[T.onset] Cov,-1.102,-1.037,0.300,n.s.,-3.185,0.981
event[T.saccade] x channel_type[T.onset] Cov,1.476,1.478,0.140,n.s.,-0.482,3.433
channel_type[T.onset] Var,2.073,1.643,0.100,n.s.,-0.400,4.546


#### (1B) Check Residual Normality
Verify that the model's residuals are normally distributed, to justify the use of a nested linear mixed effects (NLME) model.

We use the `D’Agostino-Pearson` and `Shapiro-Wilk` tests to check for normality, and also plot the distribution of the residuals for visual inspection.

In [29]:
residuals = global_results.resid
print("Residuals Normality Tests:\n")

skew, kurt = st.skew(residuals), st.kurtosis(residuals, fisher=True)
print(f"Residual Skewness: {skew:.3f} [Normal=0], Kurtosis: {kurt:.3f} [Normal=0]\n")

k2, p = st.normaltest(residuals)    # D'Agostino-Pearson test
output = "Data is normally distributed" if p > 0.05 else "Data is not (!) normally distributed"
print(f"D'Agostino-Pearson test: k2={k2:.3f}, p-value={p:.3f} --> {output}\n")

w, p = st.shapiro(residuals)        # Shapiro-Wilk test
output = "Data is normally distributed" if p > 0.05 else "Data is not (!) normally distributed"
print(f"Shapiro-Wilk test: W={w:.3f}, p-value={p:.3f} --> {output}\n")

and_res = st.anderson(residuals, dist="norm")

residuals_fig = px.histogram(residuals, title="Residuals Distribution (Global Model)")
residuals_fig.update_layout(
    font_family=FONT_FAMILY,
    width=800, height=450,
    paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',
    margin=dict(l=0, r=0, t=25, b=0, pad=0),
    showlegend=False,
)
residuals_fig.show()

Residuals Normality Tests:

Residual Skewness: 0.164 [Normal=0], Kurtosis: 0.553 [Normal=0]

D'Agostino-Pearson test: k2=7.912, p-value=0.019 --> Data is not (!) normally distributed

Shapiro-Wilk test: W=0.992, p-value=0.006 --> Data is not (!) normally distributed



#### (1C) Post-Hoc (Dunn's Test)
We check for pairwise differences in detectability, using the mean performance across all detectors

In [30]:
across_detector_post_hoc = posthoc_dunn(metrics, METRIC, ANNOTATOR, "mean", include_annotators=False)

print("Dunn's Test Results (Across Detectors):")
display(across_detector_post_hoc)

Dunn's Test Results (Across Detectors):


event                  fixation       saccade      
channel_type             offset onset  offset onset
event    channel_type                              
fixation offset              --   ***     ***  n.s.
         onset         0.000062    --    n.s.   ***
saccade  offset        0.000047   1.0      --   ***
         onset              1.0   0.0     0.0    --

#### (1D) Effect Sizes: Cliff's Delta
We use Cliff's delta to measure the effect size of significant simple-effect (pairwise comparisons) results from Dunn's test.
This effect size calculation is used for non-parametric tests and is based on the `cliffs-delta` python package.

In [31]:
from cliffs_delta import cliffs_delta

data = _extract_data(metrics, METRIC, ANNOTATOR, "mean", include_annotators=False)
cliffs_delta_results = across_detector_post_hoc.copy()
for i, (evnt1, chan1) in enumerate(across_detector_post_hoc.index):
    for j, (evnt2, chan2) in enumerate(across_detector_post_hoc.columns):
        if i <= j:
            continue
        if evnt1 == evnt2 and chan1 == chan2:
            continue
        scores1 = data.loc[(evnt1, chan1)].values
        scores2 = data.loc[(evnt2, chan2)].values
        d, res = cliffs_delta(scores1, scores2)
        cliffs_delta_results.loc[(evnt1, chan1), (evnt2, chan2)] = (d, res)

print("Cliff's Delta Results (Across Detectors):")
display(cliffs_delta_results)

Cliff's Delta Results (Across Detectors):


event                         fixation                              saccade  \
channel_type                    offset                 onset         offset   
event    channel_type                                                         
fixation offset                     --                   ***            ***   
         onset          (-0.91, large)                    --           n.s.   
saccade  offset        (-0.865, large)  (-0.075, negligible)             --   
         onset          (0.34, medium)         (0.97, large)  (0.91, large)   

event                        
channel_type          onset  
event    channel_type        
fixation offset        n.s.  
         onset          ***  
saccade  offset         ***  
         onset           --

In [32]:
for i, (e1, c1) in enumerate(across_detector_post_hoc.index):
    for j, (e2, c2) in enumerate(across_detector_post_hoc.columns):
        if i <= j:
            continue
        print(f"{(e1, c1)} vs {(e2, c2)}:\tp={across_detector_post_hoc.loc[(e1, c1), (e2, c2)] :.6f}\td={cliffs_delta_results.loc[(e1, c1), (e2, c2)][0] :.3f} ({cliffs_delta_results.loc[(e1, c1), (e2, c2)][1]})")

('fixation', 'onset') vs ('fixation', 'offset'):	p=0.000062	d=-0.910 (large)
('saccade', 'offset') vs ('fixation', 'offset'):	p=0.000047	d=-0.865 (large)
('saccade', 'offset') vs ('fixation', 'onset'):	p=1.000000	d=-0.075 (negligible)
('saccade', 'onset') vs ('fixation', 'offset'):	p=1.000000	d=0.340 (medium)
('saccade', 'onset') vs ('fixation', 'onset'):	p=0.000000	d=0.970 (large)
('saccade', 'onset') vs ('saccade', 'offset'):	p=0.000000	d=0.910 (large)


### (2) Within-Detector Comparison
Here we include all annotators (human and algorithmic) to check for differences in detectability between events (fixations vs. saccades) **within** each detector, controlling for channel type (onset/offset).

#### (2A) Linear Mixed Effects Model
We use the reduced model, controlling for variability between channels (onset/offset):
$$d' \sim 1 + \text{event} + (1|\text{channel})$$

In [33]:
%%capture --no-display

per_detector_results = {
    det: linear_mixed_effect(metrics, METRIC, ANNOTATOR, det, include_annotators=True)
    for det in metrics.columns.get_level_values(u.PRED_STR).unique() if det != ANNOTATOR
}

In [34]:
for det in sorted(per_detector_results.keys(), key=lambda dett: LABELER_PLOTTING_CONFIG[dett][0]):
    res = per_detector_results[det]
    is_significant = res.pvalues['event[T.saccade]'] <= 0.05
    print(f"################################")
    print(f"Detector: {det}")
    print(f"Significant difference between fixations vs. saccades?\t{is_significant}")
    display(res.summary())
    if det != "remodnav":   # avoid line space in the final iteration
        print("\n\n")

################################
Detector: MN
Significant difference between fixations vs. saccades?	False


<class 'statsmodels.iolib.summary2.Summary'>
"""
                    Mixed Linear Model Regression Results
=============================================================================
Model:                     MixedLM        Dependent Variable:        d_prime 
No. Observations:          56             Method:                    REML    
No. Groups:                2              Scale:                     0.4401  
Min. group size:           28             Log-Likelihood:            -60.2315
Max. group size:           28             Converged:                 Yes     
Mean group size:           28.0                                              
-----------------------------------------------------------------------------
                                    Coef.  Std.Err.   z   P>|z| [0.025 0.975]
-----------------------------------------------------------------------------
Intercept                            3.848    1.069 3.599 0.000  1.752  5.943
event[T.saccade]                     0.696    1.788 0.389 0.697 -2.809  4.201
channel_type Var                     2.255                                   
channel_type x event[T.saccade] Cov -3.776                                   
event[T.saccade] Var                 6.334                                   
=============================================================================

"""




################################
Detector: ivt
Significant difference between fixations vs. saccades?	False


<class 'statsmodels.iolib.summary2.Summary'>
"""
                    Mixed Linear Model Regression Results
=============================================================================
Model:                    MixedLM        Dependent Variable:        d_prime  
No. Observations:         80             Method:                    REML     
No. Groups:               2              Scale:                     0.8516   
Min. group size:          40             Log-Likelihood:            -110.7079
Max. group size:          40             Converged:                 No       
Mean group size:          40.0                                               
-----------------------------------------------------------------------------
                                    Coef.  Std.Err.   z   P>|z| [0.025 0.975]
-----------------------------------------------------------------------------
Intercept                            2.988    1.319 2.265 0.024  0.403  5.574
event[T.saccade]                     0.229    2.569 0.089 0.929 -4.806  5.263
channel_type Var                     3.438                                   
channel_type x event[T.saccade] Cov -6.709                                   
event[T.saccade] Var                13.109                                   
=============================================================================

"""




################################
Detector: ivvt
Significant difference between fixations vs. saccades?	False


...

<class 'statsmodels.iolib.summary2.Summary'>
"""
                     Mixed Linear Model Regression Results
===============================================================================
Model:                    MixedLM         Dependent Variable:         d_prime  
No. Observations:         80              Method:                     REML     
No. Groups:               2               Scale:                      0.6462   
Min. group size:          40              Log-Likelihood:             -100.5909
Max. group size:          40              Converged:                  Yes      
Mean group size:          40.0                                                 
-------------------------------------------------------------------------------
                                     Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------
Intercept                             3.678    1.379  2.667 0.008  0.975  6.381
event[T.saccade]                     -0.616    4.150 -0.149 0.882 -8.750  7.517
channel_type Var                      3.772                                    
channel_type x event[T.saccade] Cov -11.384                                    
event[T.saccade] Var                 34.376                                    
===============================================================================

"""




################################
Detector: nh
Significant difference between fixations vs. saccades?	False


<class 'statsmodels.iolib.summary2.Summary'>
"""
                    Mixed Linear Model Regression Results
==============================================================================
Model:                     MixedLM        Dependent Variable:        d_prime  
No. Observations:          80             Method:                    REML     
No. Groups:                2              Scale:                     0.7857   
Min. group size:           40             Log-Likelihood:            -104.9733
Max. group size:           40             Converged:                 Yes      
Mean group size:           40.0                                               
------------------------------------------------------------------------------
                                    Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------------------
Intercept                            2.019    0.141 14.313 0.000  1.743  2.296
event[T.saccade]                     0.020    0.200  0.099 0.921 -0.373  0.413
channel_type Var                     0.001                                    
channel_type x event[T.saccade] Cov -0.001                                    
event[T.saccade] Var                 0.002    0.143                           
==============================================================================

"""




################################
Detector: remodnav
Significant difference between fixations vs. saccades?	False


<class 'statsmodels.iolib.summary2.Summary'>
"""
                   Mixed Linear Model Regression Results
============================================================================
Model:                    MixedLM        Dependent Variable:        d_prime 
No. Observations:         80             Method:                    REML    
No. Groups:               2              Scale:                     0.2743  
Min. group size:          40             Log-Likelihood:            -66.1078
Max. group size:          40             Converged:                 Yes     
Mean group size:          40.0                                              
----------------------------------------------------------------------------
                                    Coef. Std.Err.   z   P>|z| [0.025 0.975]
----------------------------------------------------------------------------
Intercept                           1.226    0.261 4.693 0.000  0.714  1.737
event[T.saccade]                    0.512    0.457 1.120 0.263 -0.384  1.407
channel_type Var                    0.123    1.048                          
channel_type x event[T.saccade] Cov 0.219    0.628                          
event[T.saccade] Var                0.390    2.599                          
============================================================================

"""

#### (2B) Within Detector Post-Hoc (Dunn's Test)

In [35]:
posthoc_results = {
    det: posthoc_dunn(metrics, METRIC, ANNOTATOR, det, include_annotators=True)
    for det in metrics.columns.get_level_values(u.PRED_STR).unique() if det != ANNOTATOR
}

for det in sorted(posthoc_results.keys(), key=lambda dett: LABELER_PLOTTING_CONFIG[dett][0]):
    print(f"################################\nDetector: {det}")
    is_significant = per_detector_results[det].pvalues['event[T.saccade]'] <= 0.05
    if is_significant:
        display(posthoc_results[det])
    else:
        print("Main Analysis Not Significant!")
    if det != "remodnav":   # avoid line space in the final iteration
        print("\n\n")

################################
Detector: MN
Main Analysis Not Significant!



################################
Detector: ivt
Main Analysis Not Significant!



################################
Detector: ivvt
Main Analysis Not Significant!



################################
Detector: idt
Main Analysis Not Significant!



################################
Detector: idvt
Main Analysis Not Significant!



################################
Detector: engbert
Main Analysis Not Significant!



################################
Detector: nh
Main Analysis Not Significant!



################################
Detector: remodnav
Main Analysis Not Significant!


#### (2C) Within Detector Alternative Analysis :: Use ART-ANOVA
**ART** = Aligned Rank Transformation - a non-parametric method for conducting a 2*2 ANOVA (fixation/saccade * onset/offset) on the ranks of the data.

article:
https://dl.acm.org/doi/pdf/10.1145/1978942.1978963
https://depts.washington.edu/acelab/proj/art/index.html - more detailed explanation

implementation:
https://github.com/Palpatineli/artpy

In [49]:
# TODO

## Summary Figure

In [36]:
W, H = 900, 750
yaxis_title = dict(text=r'$d^{\prime}$', font=AXIS_LABEL_FONT, standoff=AXIS_LABEL_STANDOFF)
FILE_NAME_FORMAT = "supp_fig_I%d-%s"

In [37]:
fig_ra = multiple_bar_plots(metrics, METRIC, "RA", show_other_annotator=True)
fig_ra.update_layout(
    title=None,
    width=W, height=H,

    # update yaxis titles:
    yaxis_title=yaxis_title, yaxis2_title=yaxis_title, yaxis6_title=yaxis_title,
)

fig_ra.write_image(os.path.join(FIGURES_DIR, f"{FILE_NAME_FORMAT % (1, "RA")}.png"), scale=3)
# fig_ra.write_json(os.path.join(FIGURES_DIR, f"{NAME}.json"), indent=4)
fig_ra.show()

In [38]:
fig_mn = multiple_bar_plots(metrics, METRIC, "MN", show_other_annotator=True)
fig_mn.update_layout(
    title=None,
    width=W, height=H,

    # update yaxis titles:
    yaxis_title=yaxis_title, yaxis2_title=yaxis_title, yaxis6_title=yaxis_title,
)

fig_mn.write_image(os.path.join(FIGURES_DIR, f"{FILE_NAME_FORMAT % (2, "MN")}.png"), scale=3)
# fig_mn.write_json(os.path.join(FIGURES_DIR, f"{NAME}.json"), indent=4)
fig_mn.show()